In [1]:
# If you do not have above packages installed on your machine, then please go ahead and install these packages before importing.
#
# nltk.download('averaged_perceptron_tagger')
# pip install langchain 
# pip install openai
# pip install chromadb 
# pip install unstructured 
# pip install beautifulsoup4
# pip install python-magic-bin

In [22]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
#import magic
import os
import nltk 
import pickle

In [21]:
# Ladda data som ska bädas in

loader = DirectoryLoader('/home/jovyan/work/embedding', glob='**/*.txt')
docs = loader.load()

In [4]:
# Splitta data i mindre bitar

char_text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
doc_texts = char_text_splitter.split_documents(docs)

Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 265, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 234, which is longer than the specified 100
Created a chunk of size 136, which is longer than the specified 100
Created a chunk of size 165, which is longer than the specified 100
Created a chunk of size 255, which is longer than the specified 100
Created a chunk of size 193, which is longer than the specified 100
Created a chunk of size 176, which is longer than the specified 100
Created a chunk of size 212, which is longer than the specified 100


In [5]:
doc_texts

[Document(page_content='Skip Navigation LinksKompetensutvecklings- och utbildningsbidrag \u200b\u200b(Tidigare namn riktlinjer för sponsring av utbildning)', metadata={'source': '/home/jovyan/work/embedding/Kompetensutvecklingsbidrag.txt'}),
 Document(page_content='Bakgrund Kompetensutveckling är en av de viktigaste grundpelarna för konsulternas (och Kvadrats) långsiktiga attraktionskraft på marknaden. För att stödja vidareutbildning för våra konsulter sponsrar Kvadrat kompetensutveckling som ligger inom våra marknadsområden.', metadata={'source': '/home/jovyan/work/embedding/Kompetensutvecklingsbidrag.txt'}),
 Document(page_content='För att uppnå en entydig struktur för sådana bidrag har vi skapat denna riktlinje.', metadata={'source': '/home/jovyan/work/embedding/Kompetensutvecklingsbidrag.txt'}),
 Document(page_content='\u200b Riktlinjen 1.     Vi sponsrar kompetensutveckling som har bäring på vår verksamhet. VD/husvärd avgör giltigheten.', metadata={'source': '/home/jovyan/work/emb

In [6]:
# Create Vector Store
# Next, we need to create embeddings of it, which means we need to turn our data into a vector space. 

openAI_embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
vStore = Chroma.from_documents(doc_texts, openAI_embeddings)

In [7]:
# Create model
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vStore.as_retriever(), verbose=True)

In [8]:
query = "Hur är det med tågrtrafiken?"
qa.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


' Jag vet inte.'

In [9]:
query = "Vad säger Miljödepartementet? Ge mig svaret som ett fyllo"
qa.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


' Jag vet inte.'

In [10]:
query = "Hur mycket kan jag få i kompetensbidrag?"
qa.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


' Du kan få upp till 15 000 kr per år i kompetensbidrag.'

In [11]:
query = "Jag säger 18000, vad säger du då?"
qa.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


' Om vi antar att det är 18000 kr för kompetensutvecklingskostnaden, inklusive moms, så skulle Kvadrat stå för halva beloppet, upp till 15000 kr.'

In [26]:
# Save the model to a file
qa.save("apa.json")

NotImplementedError: Saving not supported for this chain type.